<a href="https://colab.research.google.com/github/MaitriKhanna/Research-Assisstant/blob/main/Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Building a Question-Answer System with BERT for Wikipedia


#Setting up virtual enviornment

- Using "venv" on our workhorse machine
-PyTorch build that includes CUDA support 
-Transformers library
-Wikipedia library for easy access and parsing of wikipedia pages.

In [ ]:
!pip install torch  torchvision -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install wikipedia

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 2.0MB 18.8MB/s 
     |████████████████████████████████| 870kB 43.8MB/s 
     |████████████████████████████████| 3.2MB 42.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=0dd71f8d35f0e247c79b7f166cadb6ffe28e3defb2e722bebf693440b47326ef
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=67c91c619a8ca36e2c61f37d6f67c4fc6b9bf29723bb8b21c1d08cfa446503d6
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


#Hugging Face Transformers
-Provides state-of-the-art general-purpose architectures for NLU and NLG

-Host pre-trained models

-Interoperability between PyTorch and Tensorflow 2.0

-We are using **Autoclasses** which serve as a wrapper around any base transformer class

#Fine Tuning the model

Downloading the SQuAD dataset

In [ ]:
# set path with magic
%env DATA_DIR=./data/squad 

# download the data
def download_squad(version=1):
    if version == 1:
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
    else:
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
            
download_squad(version=2)

env: DATA_DIR=./data/squad
--2021-04-04 09:48:02--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘./data/squad/train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   264MB/s    in 0.2s    

2021-04-04 09:48:03 (264 MB/s) - ‘./data/squad/train-v2.0.json’ saved [42123633/42123633]

--2021-04-04 09:48:03--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/js

Using a pre-fine-tuned model from the Hugging Face repository

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# executing these commands for the first time initiates a download of the 
# model weights to ~/.cache/torch/transformers/
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2") 
model = AutoModelForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

Trying out the model

Tokenize the Input--> obtain model scores--> get the answer span


In [ ]:
!pip install pdfminer.six

     |████████████████████████████████| 5.6MB 17.6MB/s 
     |████████████████████████████████| 3.2MB 37.7MB/s 


In [ ]:
text=""

In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
sample_file= '/content/gdrive/MyDrive/Colab Notebooks/Research assisstant/atmosphere.pdf'

In [ ]:
for page_layout in extract_pages(sample_file):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            text+=element.get_text()

In [ ]:
question = "What is atmosphere"

context = text
# 1. TOKENIZE THE INPUT
# note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for 
# exploration but you cannot feed that into a model. 
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. OBTAIN MODEL SCORES
# the AutoModelForQuestionAnswering class includes a span predictor on top of the model. 
# the model returns answer start and end scores for each word in the text
answer = model(**inputs)
answer_start_scores = answer[0]
answer_end_scores = answer[1]
answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

# 3. GET THE ANSWER SPAN
# once we have the most likely start and end tokens, we grab all the tokens between them
# and convert tokens back to words!
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'the thin envelope of gas molecules surrounding the Earth'

##QA on Wikipedia Pages

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia as wiki
import pprint as pp

question = 'What is atmosphere?'

results = wiki.search(question)
print("Wikipedia search results for our question:\n")
pp.pprint(results)

page = wiki.page(results[0])
text = page.content
print(f"\nThe {results[0]} Wikipedia article contains {len(text)} characters.")

Wikipedia search results for our question:

['Atmosphere',
 'Atmosphere of the Moon',
 'Atmosphere of Earth',
 'Atmosphere of Venus',
 'Atmosphères',
 'Extraterrestrial atmosphere',
 'Reducing atmosphere',
 'Atmosphere of Pluto',
 'Greenhouse effect',
 'Atmosphere of Titan']

The Atmosphere Wikipedia article contains 11439 characters.


In [ ]:
inputs = tokenizer.encode_plus(question, text, return_tensors='pt')
print(f"This translates into {len(inputs['input_ids'][0])} tokens.")

This translates into 2513 tokens.


##Tokenizers
It takes the input as text and return tokens. Here, we are using BERT model which can only accept 512 tokens at once. 
Therefore, we have tp split our input into chunks and each chunk must not exceed 512 tokens in total.

Chunk format-
[CLS] question tokens [SEP] context tokens [SEP]

In [ ]:
# time to chunk!
from collections import OrderedDict

# identify question tokens (token_type_ids = 0)
qmask = inputs['token_type_ids'].lt(1)
qt = torch.masked_select(inputs['input_ids'], qmask)
print(f"The question consists of {qt.size()[0]} tokens.")

chunk_size = model.config.max_position_embeddings - qt.size()[0] - 1 # the "-1" accounts for
# having to add a [SEP] token to the end of each chunk
print(f"Each chunk will contain {chunk_size - 2} tokens of the Wikipedia article.")

# create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
chunked_input = OrderedDict()
for k,v in inputs.items():
    q = torch.masked_select(v, qmask)
    c = torch.masked_select(v, ~qmask)
    chunks = torch.split(c, chunk_size)

    for i, chunk in enumerate(chunks):
        if i not in chunked_input:
            chunked_input[i] = {}

        thing = torch.cat((q, chunk))
        if i != len(chunks)-1:
            if k == 'input_ids':
                thing = torch.cat((thing, torch.tensor([102])))
            else:
                thing = torch.cat((thing, torch.tensor([1])))

        chunked_input[i][k] = torch.unsqueeze(thing, dim=0)

The question consists of 6 tokens.
Each chunk will contain 503 tokens of the Wikipedia article.


In [ ]:
for i in range(len(chunked_input.keys())):
    print(f"Number of tokens in chunk {i}: {len(chunked_input[i]['input_ids'].tolist()[0])}")

Number of tokens in chunk 0: 512
Number of tokens in chunk 1: 512
Number of tokens in chunk 2: 512
Number of tokens in chunk 3: 512
Number of tokens in chunk 4: 493


Each of these chunks can now be fed to the model without causing indexing errors. We'll get an "answer" for each chunk; however, not all answers are useful, since not every segment of a Wikipedia article is informative for our question. The model will return the [CLS] token when it determines that the context does not contain an answer to the question. 

In [ ]:
def convert_ids_to_string(tokenizer, input_ids):
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids))

answer = ""

# now we iterate over our chunks, looking for the best answer from each chunk
for _, chunk in chunked_input.items():
    a = model(**chunk)
    answer_start_scores = a[0]
    answer_end_scores = a[1]
    
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    ans = convert_ids_to_string(tokenizer, chunk['input_ids'][0][answer_start:answer_end])
    
    # if the ans == [CLS] then the model did not find a real answer in this chunk
    if ans != '[CLS]':
        answer += ans + " // "
        
        
print(answer)

a layer or a set of layers of gases surrounding a planet or other material body // 


##Combining all together

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering


class DocumentReader:
    def __init__(self, pretrained_model_name_or_path='bert-large-uncased'):
        self.READER_PATH = pretrained_model_name_or_path
        self.tokenizer = AutoTokenizer.from_pretrained(self.READER_PATH)
        self.model = AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH)
        self.max_len = self.model.config.max_position_embeddings
        self.chunked = False

    def tokenize(self, question, text):
        self.inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
        self.input_ids = self.inputs["input_ids"].tolist()[0]

        if len(self.input_ids) > self.max_len:
            self.inputs = self.chunkify()
            self.chunked = True

    def chunkify(self):
        """ 
        Break up a long article into chunks that fit within the max token
        requirement for that Transformer model. 

        Calls to BERT / RoBERTa / ALBERT require the following format:
        [CLS] question tokens [SEP] context tokens [SEP].
        """

        # create question mask based on token_type_ids
        # value is 0 for question tokens, 1 for context tokens
        qmask = self.inputs['token_type_ids'].lt(1)
        qt = torch.masked_select(self.inputs['input_ids'], qmask)
        chunk_size = self.max_len - qt.size()[0] - 1 # the "-1" accounts for
        # having to add an ending [SEP] token to the end

        # create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
        chunked_input = OrderedDict()
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask)
            c = torch.masked_select(v, ~qmask)
            chunks = torch.split(c, chunk_size)
            
            for i, chunk in enumerate(chunks):
                if i not in chunked_input:
                    chunked_input[i] = {}

                thing = torch.cat((q, chunk))
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
        return chunked_input

    def get_answer(self):
        if self.chunked:
            answer = ''
            for k, chunk in self.inputs.items():
                a = self.model(**chunk)


                answer_start_scores = a[0]
                answer_end_scores = a[1]
                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1

                ans = self.convert_ids_to_string(chunk['input_ids'][0][answer_start:answer_end])
                if ans != '[CLS]':
                    answer += ans + " / "
            return answer
        else:
            a = self.model(**self.inputs)


            answer_start_scores = a[0]
            answer_end_scores = a[1]      
            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
        
            return self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])

    def convert_ids_to_string(self, input_ids):
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

In [ ]:
# collapse-hide 

# to make the following output more readable I'll turn off the token sequence length warning
import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

In [ ]:
questions = [
    'What is name of son of Ambani?',
    'Why is the sky blue?',
    'How many sides does a pentagon have?'
]

reader = DocumentReader("deepset/bert-base-cased-squad2") 



for question in questions:
    print(f"Question: {question}")
    results = wiki.search(question)
    
    for i in results:
        try:
          wiki.page(i)
        except:
          continue
        page = wiki.page(i)
        break
    print(f"Top wiki result: {page}")

    text = page.content

    reader.tokenize(question, text)
    print(f"Answer: {reader.get_answer()}")
    print()


Question: What is name of son of Ambani?


Token indices sequence length is longer than the specified maximum sequence length for this model (2634 > 512). Running this sequence through the model will result in indexing errors


Top wiki result: <WikipediaPage 'Mukesh Ambani'>
Answer: [CLS] What is name of son of Ambani? [SEP] Mukesh Dhirubhai Ambani ( born 19 April 1957 ) is an Indian billionaire businessman, and the chairman, managing director, and largest shareholder of Reliance Industries Ltd. ( RIL ), a Fortune Global 500 company and India's most valuable company by market value. He is the richest person in Asia with a net worth of US $ 83. 1 billion and as of 14th March 2021, the 10th richest in the world. = = Early life = = Mukesh Dhirubhai Ambani was born on 19 April 1957 in the British Crown colony of Aden ( present - day Yemen ) to Dhirubhai Ambani and Kokilaben Ambani. He has a younger brother Anil Ambani / Dhirubhai / Anand Ambani / 

Question: Why is the sky blue?
Top wiki result: <WikipediaPage 'Diffuse sky radiation'>
Answer: Rayleigh scattering / 

Question: How many sides does a pentagon have?
Top wiki result: <WikipediaPage 'The Pentagon'>
Answer: five / 

